São **18 cadeiras** dividas em **8 módulos**
- Cadeiras de 15 horas: UAB000, UAB015, UAB002, UAB005, UAB006, UAB004, UAB014, UAB007, UAB008, UAB009, UAB011, UAB016
- Cadeiras de 30 horas: UAB001, UAB003, UAB012, UAB018, UAB010, UAB013

Notas no intervalo de 0 a 10

A prova tem peso 4 e as atividades e exercícios realizados durante a disciplina têm peso 6.

Cada módulo possui 2 ou 3 cadeiras com suas respectivas notas das atividades e a nota da prova do módulo (questionário). A nota total da categoria (nota do módulo) é calculada por uma media ponderada das notas  do módulo da forma: 

>Nota total = 30% Cadeira 1 + 30% Cadeira 2 + 40% Questionário

ou

>Nota total = 20% Cadeira 1 + 20% Cadeira 2 + 20% Cadeira 3 + 40% Questionário

Dependendo do número de cadeiras do módulo. As provas substitutivas de cada módulo já se encontram alteradas na coluna da prova padrão. O total do curso é calculado usando uma média entre as notas totais de cada módulo.

Retirar as linhas nulas???

Módulo 1:
- UAB 000 = Disciplina 30306 - Treinamento Inicial Moodle
- UAB 015 = Disciplina 30502 - Uso de email

Módulo 2:
- UAB 001 = Disciplina 30544 - Introdução ao Computador
- UAB 002 = Disciplina 30320 - Operação de Computadores

Módulo 3:
- UAB 003 = Disciplina 30334 - Editores de Texto
- UAB 005 = Disciplina 30362 - Editores de Apresentação
- UAB 006 = Disciplina 30376 - Editores de Imagem

Módulo 4:
- UAB 004 = Disciplina 30348 - Planilhas eletronicas
- UAB 014 = (?)Disciplina 30488 - Ferramentas de Apoio

Módulo 5:
- UAB 007 = Disciplina 30390 - Internet e Web
- UAB 008 = Disciplina 30404 - Riscos e Danos
- UAB 009 = Disciplina 30418 - Edição de Páginas Web

Módulo 6:
- UAB 011 = Disciplina 30446 - Mídias e sua utilização
- UAB 012 = Disciplina 30460 - Ferramentas de Comunicação

Módulo 7:
- UAB 010 = Disciplina 30432 - Busca de Recursos educativos e informações
- UAB 013 = Disciplina 30474 - Softwares Educativos


Módulo 8: (sem tabela de notas)
- UAB 016 =
- UAB 018 =







In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


def DefineConceito(df,atividadade):
    A,B,C,D,Z= [],[],[],[],[]
    index = 0
    for nota in df[atividadade]:
        if nota >= 9.0:
            A.append(df.iloc[index,0])
        elif nota >= 7.5:
            B.append(df.iloc[index,0])
        elif nota >= 6.0:
            C.append(df.iloc[index,0])
        elif nota > 0.0:
            D.append(df.iloc[index,0])
        else :
            Z.append(int((df.iloc[index,0])[6:]))
        index+=1
    return A,B,C,D,Z

def GeraGraficoConceitos (A,B,C,D,disciplina,index):
    
    plt.figure(figsize=(6,6))
    plt.title(disciplina.partition('.')[0])
    plt.xlabel('Conceito')
    plt.ylabel('Número de Alunos')
    plt.bar(["A","B","C","D"],[len(A),len(B),len(C),len(D)],color=['green', 'blue','yellow','red'])
    

def GeragraficoMedDp (df):
    dp,media,variacao=CoenficienteVariacao (df)
    plt.figure(figsize=(12, 12))
    plt.subplot(311)
    plt.title("Desvio Padrão")
    dp.plot(kind='bar')
    plt.subplot(312)
    plt.title("Média")
    media.plot(kind='bar')
    plt.subplot(313)
    plt.title("Coeficiente de Variação")
    variacao.plot(kind='bar')
    
def CriaXLSX (df):
    writer = pd.ExcelWriter('MediasAjustadas.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()

def CalculaMediaDasAtividades(df,atividades):
    medias=[]
    for index in range(1,len(atividades)):
        medias.append(round(df[atividades[index]].mean(),2))
    print(medias)
    
def DeletaLinha(df,atividade):
    index=0
    alunosnulos=[]
    df.iloc[:,0]=df.iloc[:,0].astype(str)
    for media in df[atividade]:
        if (df.iloc[index,0])[0:5] != "ALUNO" :
            if (df.iloc[index,0])[0:4] != "Peso" :
                df = df.drop(df.index[index])
                index-=1
        elif int((df.iloc[index,0])[6:]) > 348:
            df = df.drop(df.index[index])
            index-=1
        index+=1
    df.index=range(len(df[atividade]))
    return df

def AjustaMedia (df,atividade,numdeatividades):
    index = 0
    pesos = df.iloc[0,1:numdeatividades]
    for media in df[atividade]:
        mediacalculada =round( np.dot(pesos , df.iloc[index,1:(numdeatividades)]),2)
        if (media != mediacalculada and (abs(media-mediacalculada)) > 0.5): #Considerando arredontamentos
            #print(index,media,"-->",mediacalculada)
            df.at[index, atividade] = mediacalculada
        index+=1
    df = df.drop(0)
    return df

def CoenficienteVariacao (df):
    
    var=df.var(axis=0)
    media= df.mean(axis=0)
    dp = var**(1/2)
    return dp,media,dp/media

def AtividadesNaoFeitas (df):
    
    alunoscomzero = []
    for index in range(len(df.index)):
        if (df.iloc[index] == 0).astype(int).sum()>0:
            alunoscomzero.append(index+1)
    return alunoscomzero

def CalculaPorcentagem(amostra,total):
    
    porcentagem = amostra/total *100
    print (round(porcentagem,2),"%")
    
def LeDisciplinas (nomearquivo):
    
    nomes = []
    linhas = (linha.rstrip('\n') for linha in open(nomearquivo,encoding = "ISO-8859-1"))
    for disciplina in linhas:
        nomes.append(disciplina+".xlsx")
    return nomes

def RenomeiaColunas(df,numdeatividades):
    
    nomes=[]
    nomes.append(df.columns[0])
    for colunas in range(numdeatividades-1):
        nomes.append("atividade" + str(colunas))
    nomes.append(df.columns[numdeatividades])
    df.columns = nomes
    
def DeletaNovosAlunos (df):
    
    novosalunos = []
    index = 0
    df.iloc[:,0]=df.iloc[:,0].astype(str)
    for nome in df.iloc[:,0]:
        numero = int(nome[6:])
        if numero > 348:
            df = df.drop(df.index[index])
            index-=1
        index+=1
    df.index=range(len(df.iloc[:,0]))
    return df

def AlunoNaoTotal (nomes):
    
    todosalunos = {}
    for disciplina in nomes:
        #print("-------------------" + disciplina + "--------------------")
        df = pd.read_excel(disciplina)
        disciplina = disciplina[:16]
        atividades = df.columns
        numdeatividades = len(atividades)-1
        mediacadeira = atividades[len(atividades)-1]

        novosalunos=AchaNovosAlunos(df)
        for aluno in novosalunos:
            if aluno in todosalunos:
                todosalunos[aluno].append(disciplina)
            if aluno not in todosalunos:
                todosalunos[aluno] = [disciplina]
        

    for aluno in todosalunos:
        print(aluno,":",todosalunos[aluno])
def AjeitaTabela (disciplina):
    df = pd.read_excel(disciplina)
    atividades = df.columns
    numdeatividades = len(atividades)-1
    mediacadeira = atividades[len(atividades)-1]
    
    df = DeletaLinha(df,mediacadeira)
    df = AjustaMedia(df,mediacadeira,numdeatividades)
    
    return df,numdeatividades,mediacadeira

def AlunosNaoConcluintes (df,Z,alunosdesistentes):
    alunodesistiu = []
    for aluno in Z:
        if aluno not in alunosdesistentes:
            alunodesistiu.append(aluno)
            alunosdesistentes.append(aluno)
    print(alunodesistiu,Z)
    return alunosdesistentes
    
pdf = PdfPages("ConceitosXNotas.pdf")
nomes = LeDisciplinas ("Nomes das Disciplinas.txt")
alunosdesistentes = []
with PdfPages('ConceitosXNotas.pdf') as pdf:
    for i in range(16):
        disciplina = nomes[i]
        df,numdeatividades,mediacadeira = AjeitaTabela(disciplina)

        A,B,C,D,Z = DefineConceito(df,mediacadeira)
        GeraGraficoConceitos (A,B,C,D,disciplina,i)

        pdf.savefig()  
        plt.close()




# Análise 
92.11 % dos alunos que tiraram B na cadeira UAB000 tiraram zero em ao menos uma atividade

alunos que não tem registro em todas as cadeiras = 

['ALUNO_400', 'ALUNO_401', 'ALUNO_402', 'ALUNO_403', 'ALUNO_404', 'ALUNO_405', 'ALUNO_406', 'ALUNO_407', 'ALUNO_408', 'ALUNO_409', 'ALUNO_410', 'ALUNO_411', 'ALUNO_412', 'ALUNO_413', 'ALUNO_414', 'ALUNO_415', 'ALUNO_416', 'ALUNO_417', 'ALUNO_419', 'ALUNO_420', 'ALUNO_421', 'ALUNO_424', 'ALUNO_425', 'ALUNO_426', 'ALUNO_427', 'ALUNO_428', 'ALUNO_429', 'ALUNO_430', 'ALUNO_431', 'ALUNO_432', 'ALUNO_433', 'ALUNO_434', 'ALUNO_436', 'ALUNO_437', 'ALUNO_438', 'ALUNO_439', 'ALUNO_440', 'ALUNO_441', 'ALUNO_443', 'ALUNO_451', 'ALUNO_452', 'ALUNO_453']